In [1]:
from ks_api_client import ks_api
import requests
import pandas as pd
from datetime import date
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
import time
import os
import numpy as np
import codecs
from selenium import webdriver
# Login

client = ks_api.KSTradeApi(access_token = "6c338abe-f147-37a5-b599-e6c07372b1d0", userid = "HK15011998", \
                consumer_key = "bjDBJyLrS0WKR1wrmBwBECC0DGka", ip = '192.168.29.140', app_id = "")

client.login(password = "FN3hYt18n6")
clear_output()

In [2]:
# Enter OTP in access_code

client.session_2fa(access_code = "5475")
clear_output()
print('Logged in successfully')

Logged in successfully


In [3]:
# Getting today's year, month and day to prepare the url 

today = date.today()
year = today.year
month = today.month
day = today.day

# We convert month from single digit to double digit. For example, 7 to 07.

if day < 10:
    day = '0' + str(day)
else:
    pass

if month < 10:
    month = '0' + str(month)
else:
    pass

# Check if the format for date is correct or not

#print('Year: ' + str(year))
#print('Month: ' + str(month))
#print('Day: ' + str(day) + '\n')

# Creating the URL

cashurl = 'https://preferred.kotaksecurities.com/security/production/TradeApiInstruments_Cash_' + str(day) + '_' + str(month) + '_' + str(year) + '.txt'
fnourl = 'https://preferred.kotaksecurities.com/security/production/TradeApiInstruments_fno_' + str(day) + '_' + str(month) + '_' + str(year) + '.txt'

# Manually check teh URL if there is some issue

#print(cashurl)
#print(fnourl)

# Requesting URL

cash = requests.get(cashurl)
fno = requests.get(fnourl)

# Creating text file and writing data to it

cashfile = open("Cash.txt","w")
fnofile = open('FNO.txt', "w")
cashfile.write(cash.text)
fnofile.write(fno.text)
cashfile.close()
fnofile.close()

#print(fnourl)


In [4]:
def fnofilter(fnodf, fotype, instrutype, cashdf):
    
    fnodf = fnodf.loc[fnodf['instrumentType'] == instrutype]
    
    # Asking the actual Ticker name
    
    print('List of instrument names: ')
    print(fnodf['instrumentName'].unique())
    ticker = input('Enter the ticker: ')
    fnodf = fnodf.loc[fnodf['instrumentName'] == ticker]
    
    # Expiry
    
    print('List of expiries: ')
    print(fnodf['expiry'].unique())
    
    expiry = input('Enter the expiry here. Make sure you enter the exact value, or this won\'t work. ')
    fnodf = fnodf.loc[fnodf['expiry'] == expiry]
    
    # Segregating futures and options
    
    if fotype == 'option':

        print('List of available strike prices: ')
        print(fnodf['strike'].unique())
        strike = float(input('Enter the strike price: '))
        fnodf = fnodf.loc[fnodf['strike'] == strike]
        print('Lot size: ' + str(fnodf['lotSize'].iloc[0]))
        print('Expiry: ' + str(fnodf['expiry'].iloc[0]) + '\n')
        
        # Printing both CE & PE
        
        for i in range(2):
            print('Option Type: ' + str(fnodf['optionType'].iloc[i]))
            print('Token: ' + str(fnodf['instrumentToken'].iloc[i]))
            print('LTP: ' + client.quote(instrument_token = int(fnodf['instrumentToken'].iloc[i]))['success'][0]['ltp'])
            
        
    elif fotype == 'futures':
        
        print('Instrument Name: ' + fnodf['instrumentName'].iloc[0])
        print('Token: ' + str(fnodf['instrumentToken'].iloc[0]))
        print('LTP: ' + client.quote(instrument_token = int(fnodf['instrumentToken'].iloc[0]))['success'][0]['ltp'])
        print('Expiry: ' + str(fnodf['expiry'].iloc[0]))

In [5]:
# Reading the data in Pandas and making a data frame

cashdf = pd.read_csv('Cash.txt', delimiter='|')
fnodf = pd.read_csv('FNO.txt', delimiter='|')

# Skip both of the next cells

In [6]:
# Find tokens using this

category = input('Cash or FNO?\n')

if category == 'Cash' or category == 'CASH' or category == 'cash' or category == 1:
    
    try:
        cashdf = cashdf.loc[cashdf['segment'] == 'CASH']
        exchange = input('NSE or BSE?: ')
        cashdf = cashdf.loc[cashdf['exchange'] == exchange]
        stock = input('Enter the stock ticker: ')

        stocklist = cashdf.loc[cashdf['instrumentName'] == stock]
        token = int(stocklist['instrumentToken'].iloc[0])
        print(f'Token for {stock} is {token}')
    except:
        print('Check stock ticker or run the code from the start.')
        
elif category == 'FNO' or category == 'fno' or category == 'Fno' or category == 'Derivatives' or category == 2:
    
    try:
        # First filter: Currecncy, Commodity, or FNO
        
        focategory = int(input('COMMODITY, CDS or FO? Enter 1, 2 or 3'))
        if focategory == 1:
            focategory = 'COMMODITY'
        elif focategory == 2:
            focategory = 'CDS'
        elif focategory == 3:
            focategory = 'FO'
        fnodf = fnodf.loc[fnodf['segment'] == focategory]
        
        # Asking for futures or options
        
        if focategory == 'FO':
            instrutype = int(input('Index Futures[FI], Index Options[OI], Stock Futures[FS], Stock Options[OS]. Enter 1, 2, 3 or 4 '))
            if instrutype == 1:
                instrutype = 'FI'
            elif instrutype == 3:
                instrutype = 'FS'
            elif instrutype == 2:
                instrutype = 'OI'
            else:
                instrutype = 'OS'
            if instrutype == 'FI' or instrutype == 'FS':
                fnofilter(fnodf, 'futures', instrutype, cashdf)
            else:
                print(instrutype)
                fnofilter(fnodf, 'option', instrutype, cashdf)
                

        elif focategory == 'COMMODITY':
            instrutype = int(input('Commodity Futures[CF] or Options[CO] ? Enter 1 or 2 '))
            if instrutype == 1:
                instrutype = 'CF'
            else:
                instrutype = 'CO'
            if instrutype == 'CF' or instrutype == 'cf':
                fnofilter(fnodf, 'futures', instrutype, cashdf)
            else:
                fnofilter(fnodf, 'option', instrutype, cashdf)
        elif focategory == 'CDS':
            instrutype = int(input('Currency Futures[FC] or Options[OC] ? Enter 1 or 2 '))
            if instrutype == 1:
                instrutype = 'FC'
            else:
                instrutype = 'OC'
            if instrutype == 'FC' or instrutype == 'fc':
                fnofilter(fnodf, 'futures', instrutype, cashdf)
            else:
                fnofilter(fnodf, 'option', instrutype, cashdf)
        #tickerlist = tickerlist.loc[fnodf['instrumentType'] == instrutype]

    except Exception as e:
        print(e)
        print('Check stock ticker or run the code from the start.')

KeyboardInterrupt: Interrupted by user

In [ ]:
# Constructing P/L

#bnfce = 43158
#bnfpe = 43103
niftyce = 34509
niftype = 34510
cestrike = 15600
pestrike = 15600
niftyfut = 53473

# Entry prices

#bnfceentry = 66.85
#bnfpeentry = 82.15
niftyceentry = 72
niftypeentry = 16.3

bookedprofit = 12.4

print('Nifty entries: Ce: ' + str(niftyceentry) + ' Pe : ' + str(niftypeentry))

# P/L

while True:
    nifty = (niftyceentry - float(client.quote(instrument_token = niftyce)['success'][0]['ltp'])) + (niftypeentry - float(client.quote(instrument_token = niftype)['success'][0]['ltp']) + bookedprofit)
    print('Fut: ' + client.quote(instrument_token = niftyfut)['success'][0]['ltp'] + '. LTPs: Ce: ' + client.quote(instrument_token = niftyce)['success'][0]['ltp'] + ' Pe: ' + client.quote(instrument_token = niftype)['success'][0]['ltp'] + '. {} CE P/L: {:.2f}({:.2f})'.format(cestrike, (niftyceentry - float(client.quote(instrument_token = niftyce)['success'][0]['ltp']))*75, niftyceentry - float(client.quote(instrument_token = niftyce)['success'][0]['ltp'])) + '. {} PE P/L: {:.2f}({:.2f})'.format(pestrike, (niftypeentry - float(client.quote(instrument_token = niftype)['success'][0]['ltp']))*75, niftypeentry - float(client.quote(instrument_token = niftype)['success'][0]['ltp'])) + '. P/L : {:.2f}.'.format(nifty*75), end = '\r')


# while True:
#     #clear_output()
#     #print('Banknifty entries: Ce: ' + str(bnfceentry) + ' Pe : ' + str(bnfpeentry))
#     #print('Banknifty LTPs: Ce: ' + client.quote(instrument_token = bnfce)['success'][0]['ltp'] + ' Pe: ' + client.quote(instrument_token = bnfpe)['success'][0]['ltp'])
#     print('Nifty LTPs: Ce: ' + client.quote(instrument_token = niftyce)['success'][0]['ltp'] + ' Pe: ' + client.quote(instrument_token = niftype)['success'][0]['ltp'], end = '\r')
#     #banknifty = (bnfceentry - float(client.quote(instrument_token = bnfce)['success'][0]['ltp'])) + (bnfpeentry - float(client.quote(instrument_token = bnfpe)['success'][0]['ltp']))
#     print('{} CE P/L: {:.2f}'.format(cestrike, (niftyceentry - float(client.quote(instrument_token = niftyce)['success'][0]['ltp']))*75), end = '\r')
#     print('{} PE P/L: {:.2f}'.format(pestrike, (niftypeentry - float(client.quote(instrument_token = niftype)['success'][0]['ltp']))*75), end = '\r')
#     nifty = (niftyceentry - float(client.quote(instrument_token = niftyce)['success'][0]['ltp'])) + (niftypeentry - float(client.quote(instrument_token = niftype)['success'][0]['ltp'])
#     #print('Banknifty : {}'.format(banknifty*25))
#     print('Nifty : {:.2f}'.format(nifty*75), end = '\r')
#     time.sleep(2)

In [6]:
html_template = """
<html>
<head>
<style>
body, html {{
  height: 100%;
  margin: 0;
}}

.bg {{
  /* The image used */
  background-image: url("img.jpg");

  /* Full height */
  height: 100%; 

  /* Center and scale the image nicely */
  background-position: center;
  background-repeat: no-repeat;
  background-size: cover;
  position: absolute
  filter: blur(7px);
  -webkit-filter: blur(7px);
}}

.display{{
position: fixed;
top: 8px;
width: 100%;
z-index : 9;
text-align: center;
}}

h1{{
font-family: "Lucida Handwriting", "Courier New", monospace;
font-size: 60px;
}}

body{{
font-family: monaco, monospace;
color: white;
}}
</style>
</head>
<body>


<div class="bg"></div>
<div class="display">{}{}</div>
  
</body>
</html>
"""

if today.weekday() <= 2 or today.weekday() == 4:
    stoploss = 15
else:
    stoploss = 5
    

def timecheck():
    currenttime = time.localtime()
    hour = currenttime.tm_hour
    minutes = currenttime.tm_min
    
    return hour, minutes

# Webpage Display

f = open('Live.html', 'w')


def opencheck():
    hour, minutes = timecheck()
    if hour >= 15 and minutes >= 5:
        return True
    else:
        return False
    
def dataappend(ceentry = 0, peentry = 0, highstrikeprice = 0, lowstrikeprice = 0, cepnl= 0, pepnl = 0, ceclosed = 0, peclosed = 0):
    
    data = pd.read_csv('backtested data.csv', index_col=0)
    
    #print(data)
    
    date1 = date.today()
    cepnl = cepnl*50
    pepnl = pepnl*50
    finalpl = cepnl + pepnl
    
    appenddata = {'date': [date1], 'cestrikeprice' : [highstrikeprice], 'pestrikeprice' : [lowstrikeprice], 'ceentry' : [ceentry] , 'peentry' : [peentry] , 'ceexit' : [ceclosed], 'peexit' : [peclosed] , 'cepnl' : [cepnl] , 'pepnl' : [pepnl] , 'finalpl' : [finalpl]}
    appenddata = pd.DataFrame(data = appenddata)
    livedata = pd.concat([data, appenddata])
    livedata.to_csv('backtested data.csv')
    

# the html code which will go in the file 
webprint = '<br />'

day = int(day)

if today.weekday() >=5:
    greet = '<h1>Welcome, Hrituraj</h1><h2><br />Working on weekends now, are we?<br />That\'s fantastic. Either sweat in training or bleed in battle, amirite?<br />Happy training. Just remember to take frequent breaks and to go for walks.'
else:
    greet = '<h1>Good Morning, Hrituraj</h1><h2><br />Welcome to Intraday Option Writing. <br />Today is a lovely day as any to sell options.</h2>'
    
hour, minutes = timecheck()

if hour >= 12 and hour < 17:
    greet = '<h1>Good Afternoon, Hrituraj</h1><h2><br />Welcome to Intraday Option Writing. <br />Today is a lovely day as any to sell options.</h2>'
elif hour >= 17:
    greet = '<h1>Good Evening, Hrituraj</h1><h2><br />Working late? The evening is just as lovely as you are.<br />Just stay focused and keep grinding. I know you will get there.</h2>'    
    
# writing the code into the file
f.write(html_template.format(greet, webprint))

f.close()
  
file = codecs.open("Live.html", 'r', "utf-8")

driver = webdriver.Chrome('./chromedriver')
driver.get('file:///'+os.getcwd()+'/' + 'Live.html')


def web(webprint):
    f = open('Live.html', 'w')
    f.write(html_template.format(greet, webprint))
    f.close()
    driver.refresh()


In [ ]:
# Strangle

strikeprices = range(1000,20000,50)
stratdf = fnodf.loc[(fnodf['segment'] == 'FO') & (fnodf['instrumentType'] == 'FI') & (fnodf['instrumentName'] == 'NIFTY')]
if today.day > 21:
    exp = int(input('Choose the expiry'))
else:
    exp = 0

def check():
    futtoken = int(stratdf['instrumentToken'].iloc[exp])
    ltp = float(client.quote(instrument_token = futtoken)['success'][0]['ltp'])
    
    print('{} : {:.2f}'.format(client.quote(instrument_token = futtoken)['success'][0]['stk_name'], ltp))
    high = float(client.quote(instrument_token = futtoken)['success'][0]['high_price'])
    low = float(client.quote(instrument_token = futtoken)['success'][0]['low_price'])
    average = (high+low)/2
    
    print('High: '+ str(high))# + '. Low: ' + str(low) + '. Average: ' + str(average), end = '\r')
    print('Low: ' + str(low))            
    print('Average: ' + str(average))
    time.sleep(1)
    #os.system('clear')
    clear_output()
    
    return ltp, high, low, average

def pl(high, low, webprint):
    print('Average breached. Taking position')
    webprint = webprint + '<br />Average breached. Taking position'
    for i in strikeprices:
        if high > i:
            continue
        else:
            highstrikeprice = i
            #print(highstrikeprice)
            token = fnodf.loc[(fnodf['segment'] == 'FO') & (fnodf['instrumentType'] == 'OI') & (fnodf['instrumentName'] == 'NIFTY')]
            expiry = token['expiry'].unique()[0]
            token = token.loc[(token['expiry'] == expiry) & (token['strike'] == highstrikeprice) & (token['optionType'] == 'CE')]
            token = int(token['instrumentToken'].iloc[0])
            #print(token)
            #print(client.quote(instrument_token = 34544))#['success'][0]['ltp'])
            ceentry = float(client.quote(instrument_token = token)['success'][0]['ltp'])
            print('Sold {} CE at {:.2f}'.format(highstrikeprice, ceentry))
            webprint = webprint + '<br /><strong>Sold {} CE at {:.2f}</strong>'.format(highstrikeprice, ceentry)
            web(webprint)
            break
    for i in strikeprices:
        if low > i:
            continue
        else:
            lowstrikeprice = i-50
            #print(lowstrikeprice)
            token2 = fnodf.loc[(fnodf['segment'] == 'FO') & (fnodf['instrumentType'] == 'OI') & (fnodf['instrumentName'] == 'NIFTY')]
            expiry2 = token2['expiry'].unique()[0]
            token2 = token2.loc[(token2['expiry'] == expiry2) & (token2['strike'] == lowstrikeprice) & (token2['optionType'] == 'PE')]

            #token2 = fnodf.loc[(fnodf['segment'] == 'FO') & (fnodf['instrumentType'] == 'OI') & (fnodf['instrumentName'] == 'NIFTY') & (fnodf['expiry'] == fnodf['expiry'].unique()[0]) & (fnodf['strike'] == lowstrikeprice) & (stratdf['optionType'] == 'PE')]
            token2 = int(token2['instrumentToken'].iloc[0])
            peentry = float(client.quote(instrument_token = token2)['success'][0]['ltp'])
            webprint = webprint + '<br /><strong>Sold {} PE at {:.2f}</strong>'.format(lowstrikeprice, peentry)
            print('Sold {} PE at {:.2f}'.format(lowstrikeprice, peentry))
            web(webprint)
            break
            
    pestoploss = stoploss + peentry
    cestoploss = stoploss + ceentry
    print('CE Stop Loss: {:.2f}'.format(cestoploss))
    webprint = webprint + '<br />CE Stop Loss: {:.2f}'.format(cestoploss)
    print('PE Stop Loss: {:.2f}'.format(pestoploss))    
    webprint = webprint + '<br />PE Stop Loss: {:.2f}'.format(pestoploss)
    web(webprint)
    while True:
        
        celtp = float(client.quote(instrument_token = token)['success'][0]['ltp'])
        peltp = float(client.quote(instrument_token = token2)['success'][0]['ltp'])
        
        cepnl = ceentry - celtp
        pepnl = peentry - peltp
        
        if cepnl+pepnl >= 20:
            ceclosed = celtp
            peclosed = peltp
            dataappend(ceentry, peentry, highstrikeprice, lowstrikeprice, cepnl, pepnl, ceclosed, peclosed)
            return
        
        print('{} CE: {:.2f}'.format(highstrikeprice, (cepnl)*50))
        print('{} PE: {:.2f}'.format(lowstrikeprice, (pepnl)*50))
        temp = webprint + '<br />{} CE: {:.2f}'.format(highstrikeprice, (ceentry-celtp)*50)
        temp = temp + '<br />{} PE: {:.2f}'.format(lowstrikeprice, (peentry-peltp)*50) + '<br />{} CE LTP: {}'.format(highstrikeprice, celtp) + '<br />{} PE LTP: {}'.format(lowstrikeprice, peltp)
        temp = temp + '<br />Overall: {:.2f}'.format((cepnl+pepnl)*50)
        web(temp) 
        del temp
        timebool = opencheck()
        #print(timebool)
        if timebool == True:
            ceclosed = celtp
            peclosed = peltp
            dataappend(ceentry, peentry, highstrikeprice, lowstrikeprice, cepnl, pepnl, ceclosed, peclosed)
            break
        else:
            pass
        
        
        if peltp >= pestoploss:
            peclosed = peltp
            pepnl = peentry - peclosed
            
            print('PE SL Hit at {:.2f}. Booked a loss of {:.2f}'.format(peclosed, pepnl))
            webprint = webprint + '<br />PE SL Hit at {:.2f}. Booked a loss of {:.2f}'.format(peclosed, pepnl)
            print('Only CE Side open now')
            webprint = webprint + '<br /> Only CE Side open now'
            web(webprint)
            while True:
                celtp = float(client.quote(instrument_token = token)['success'][0]['ltp'])
                cepnl = ceentry-celtp
                print('{} CE: {}'.format(highstrikeprice, (cepnl)*50))
                temp = webprint + '<br /> {} CE: {:.2f}'.format(highstrikeprice, (cepnl)*50)
                temp = temp + '<br />Overall: {:.2f}'.format((cepnl+pepnl)*50)
                web(temp)
                del temp
                timebool = opencheck()
                if celtp >= ceentry - 3 or timebool == True or cepnl + pepnl >= 20:
                    ceclosed = celtp
                    cepnl = ceentry - ceclosed
                    webprint = webprint + '<br />Call side closed at {}. No positions open.'.format(ceclosed)
                    print('Call option P/L today: {}. Put option P/L today: {}'.format(cepnl, pepnl))
                    webprint = webprint + '<br />Call option P/L today: {}. Put option P/L today: {}'.format(cepnl, pepnl)
                    print('Call side closed at {}. No positions open'.format(ceclosed))
                    web(webprint)
                    dataappend(ceentry, peentry, highstrikeprice, lowstrikeprice, cepnl, pepnl, ceclosed, peclosed)
                    return
                    
                time.sleep(1)
                clear_output()
                
        elif celtp >= cestoploss:
            ceclosed = celtp
            cepnl = ceentry - ceclosed       
            print('CE SL Hit at {:.2f}. Booked a loss of {:.2f}'.format(ceclosed, cepnl))
            webprint = webprint + '<br />CE SL Hit at {:.2f}. Booked a loss of {:.2f}'.format(ceclosed, cepnl)
            print('Only PE Side open now')
            webprint = webprint + '<br />Only PE Side open now'
            web(webprint)
            while True:
                peltp = float(client.quote(instrument_token = token2)['success'][0]['ltp'])
                pepnl = peentry - peltp
                timebool = opencheck()
                print('{} PE: {}'.format(lowstrikeprice, (pepnl)*50))
                temp = webprint + '<br /> {} PE: {:.2f}'.format(lowstrikeprice, (pepnl)*50)
                temp = temp + '<br />Overall: {:.2f}'.format((cepnl+pepnl)*50)
                web(temp)
                del temp
                if peltp >= peentry - 3 or timebool == True or cepnl + pepnl >= 20:
                    peclosed = peltp
                    pepnl = peentry - peclosed
                    webprint = webprint + '<br />Put side closed at {}. No positions open.'.format(peclosed)
                    webprint = webprint + '<br />Call option P/L today: {}. Put option P/L today: {}'.format(cepnl, pepnl)
                    print('Call side closed at {}. No positions open'.format(ceclosed))
                    web(webprint)
                    print('Put side closed at {}. No positions open.'.format(peclosed))
                    dataappend(ceentry, peentry, highstrikeprice, lowstrikeprice, cepnl, pepnl, ceclosed, peclosed)
                    return
                time.sleep(1)
                clear_output()
        time.sleep(1)
        clear_output()
starting = False
timecheck()
ltp, high, low, average = check()

if hour <=9:
    webprint = webprint + '<br />It is not 10:15 yet. You will have to wait for 10:15 before I consider taking a position. Thank you for patience.'
    web(webprint)
while True:
    hour, minutes = timecheck()
    
    if hour <= 9:
        # This loop will be in force when it is not 10 AM yet
        print('Not 10:15 Yet')
        print('Time: {}:{}'.format(hour, minutes))
        ltp, high, low, average = check()
        
        starting = False
        continue
    if minutes <= 14 and starting == False:
        # This loop will be in force when you are post 10 AM but pre 10:15
        print('Not 10:15 Yet')
        print('Time: {}:{}'.format(hour, minutes))
        ltp, high, low, average = check()
        continue
    else:
        break

    
    
    
if ltp > average:
    side = 'higher'
    webprint = webprint + '<br />Nifty Future\'s LTP is higher than the average' + '<br />I will now be checking for average breach'
    web(webprint)
    print('LTP is higher than the average')
    print('Checking for average breach')
else:
    webprint = webprint + '<br />Nifty Future\'s LTP is lower than the average' + '<br />I will now be checking for average breach'
    web(webprint)
    print('LTP is lower than the average')
    print('Checking for average breach')
    side = 'lower'    
    
while True:
    if side == 'higher':
        hour, minutes = timecheck()
        if hour >= 13:
            print('Cannot take a position after 1 PM')
            webprint = webprint + '<br />It doesn\'t seem like Nifty is in the mood for coming back to our average.<br /> It has only gone higher and hasn\'t returned and it is 1 PM already. Oh well.<br /> We will look for a position tomorrow. Until then, stay frosty, <strong>Jack the Ripper.</strong>'
            web(webprint)
            dataappend()
            break
        ltp, high, low, average = check()

        if ltp > average:
            continue
        else:
            pl(high, low, webprint)
            break

    else:
        hour, minutes = timecheck()
        if hour >= 13:
            print('Cannot take a position after 1 PM')
            webprint = webprint + '<br />It doesn\'t seem like Nifty is in the mood for coming back to our average.<br /> It has only gone lower and hasn\'t returned and it is 1 PM already. Oh well. <br />We will look for a position tomorrow. Until then, stay frosty, <strong>Jack the Ripper.</strong>' 
            web(webprint)
            dataappend()
            break
        ltp, high, low, average = check()
        if ltp < average:
            continue
        else:
            pl(high, low, webprint)
            break